# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [3]:

! pip install pandas
! pip install regex
! pip install matplotlib
! pip install nltk
! pip install sklearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 77.8 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
  Using cached numpy-1.23.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
  Using cached regex-2022.7.9-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (765 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
  Using cached matplotlib-3.5.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.3 MB)
  Using cached fonttools-4.34.4-py3-none-any.whl (944 kB)
  Using cached kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached Pillow-9.2.0-cp38-cp38-manylinux_2_28_x86_64.whl (3.2 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2


In [190]:
#import re
import nltk
import string
import unicodedata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re

from nltk.corpus import stopwords
from sklearn import model_selection, svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# EJEMPLOS:
'''
df['len_url'] = df['url'].apply(lambda x : len(x))
df['contains_subscribe'] = df['url'].apply(lambda x : 1 if "subscribe" in x else 0)
df['contains_hash'] = df['url'].apply(lambda x : 1 if "#" in x else 0)
df['num_digits'] = df['url'].apply(lambda x : len("".join(_ for _ in x if _.isdigit())) )
df['non_https'] = df['url'].apply(lambda x : 1 if "https" in x else 0)
df['num_words'] = df['url'].apply(lambda x : len(x.split("/")))

target = 'is_spam'
features = [f for f in df.columns if f not in ["url", target]]
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=0)
'''

In [191]:
# Example importing the CSV here
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [176]:
df.shape

(2999, 2)

In [177]:
df['is_spam'].value_counts()

False    2303
True      696
Name: is_spam, dtype: int64

In [175]:
df.head(5)

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


CAMBIAR LOS TRUE POR 1 Y LOS FALSE POR 0

In [197]:
# Remove non-alpha (except '.' and '/'), numbers, tags, lonely letters
clean = []
for i in range(len(df.url)) :
    desc = df['url'][i]

    # remove non-aplha
    desc = re.sub('[^a-zA-Z]', ' ', desc)

    # remove numbers
    desc = re.sub('[0-9]', '', desc)

    # remove tags
    desc = re.sub('&lt;/?.*?&gt;', ' &lt;&gt; ', desc)

    # remove some known unnecesary words (http, https, www, com)
    desc = re.sub('https|http|www|html|\scom\s|\sorg\s|\sus\s', '', desc)

    # remove lonely letters
    for i in range(5) :
        desc = re.sub('\s[a-zA-Z]\s', ' ', desc)

    clean.append(desc)

df['url'] = clean

In [180]:
df.head(30)

,url,is_spam
0,briefingday us list manageunsubscribe,True
1,hvper,True
2,briefingdaym,True
3,briefingdayn commentform,False
4,briefingdayfan,True


In [193]:
# Drop NAs & duplicates
df = df.dropna().drop_duplicates()
df = df.reset_index(inplace = False)[['url','is_spam']]
df.shape

# Sprip spaces & to lowercase
df['url'] = [entry.lower().strip() for entry in df['url']]

# is_spam True --> 1, False --> 0
df['is_spam'] = df['is_spam'].apply(lambda x: 1 if x == True else 0)

In [182]:
print(df.shape)

(2311, 2)


In [194]:
# Remove stop words

stop = stopwords.words('english')
def remove_stopwords(Message):
  if Message is not None :
    words = Message.strip().split()
    words_filtered = []
    for word in words :
      if word not in stop :
        words_filtered.append(word)
    result = ' '.join(words_filtered)
  else :
    result = None
  
  return result

In [195]:
df['url'] = df['url'].apply(remove_stopwords)

In [198]:
df['url'].str.split(expand=True).stack().value_counts()[:15]

coronavirus         159
news                107
numlock              86
stories              86
trump                85
morningbrewdaily     81
youtubewatch         77
new                  71
nytimes              69
utm                  62
covid                61
index                60
iduskbn              58
co                   56
substackp            55
dtype: int64

In [32]:
"""
def clean_text_digits(self, pattern, string):
    '''Match all digits in the string and replace them by empty string.'''
    return re.sub(pattern, '', string)
"""

hacer limpieza y clasificar (como en el notebook de NLP)

In [199]:
# Split train test models
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=13)

In [200]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [201]:
X_train.shape

(1733, 4994)

In [204]:
# Generate model
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Train Score
print(nb.score(X_train, y_train))

# Test Score
print(nb.score(X_test, y_test))

0.949798038084247
0.9256055363321799


In [205]:
nb.predict(vec.transform(['free']))

array([0])